In [1]:
"""
if x/(2 or 3) >= 2:
    return x+2
else:
    return x+5
"""

import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from QArithmetic import add
from qiskit_aer.noise import NoiseModel
from fpqs6 import fpqs_circ
import matplotlib.pyplot as plt
from QArithmetic import div, add, cadd

# Registers and circuit.
b = QuantumRegister(6)
x = QuantumRegister(6)
m = QuantumRegister(3)
anc = QuantumRegister(2)
cm = ClassicalRegister(3)
cb = ClassicalRegister(6)
cx = ClassicalRegister(6)
cc = ClassicalRegister(4)
canc = ClassicalRegister(2)
qc = QuantumCircuit(b, x, m, anc, cc)#, cx, cb, cm, canc)
qc_new = QuantumCircuit(b, x, m, anc, cc)#, cx, cb, cm, canc)

# Init
qc_new.h(x[0]) # x = from 000 000 to 000 111 
qc_new.h(x[1])
qc_new.h(x[2])

# # qc_new.x(x[0]) # x = from 000 000 to 000 111 
# qc_new.x(x[0])
# qc_new.x(x[2]) # 101 = 5

qc_new.x(b[4]) # b = 010 000 / 011 000
qc_new.h(b[3])


# Divide the numbers, so |0...0 x⟩|b 0...0⟩|0...0⟩ → |x mod b 0...0⟩|b⟩|a/b⟩
qc.barrier()
div(qc, x, b, m, 3)
qc.barrier()



# Encode |6> in the last 3 qubits of b 
# the state becomes |x mod b 000⟩|b 110⟩|0 a/b⟩ --> |x mod b 00 sign(x+6)⟩|b 110⟩|0 a/b⟩
qc.x(b[1]) # 110
qc.x(b[2]) 
# qc.x(x[0]) 
qc.barrier()

# # m+6
# # here m is using the first qubit of x to detect overflow 
add(qc, b[0:3], [m[0], m[1], m[2], x[0]], 3)
qc_inv = qc.inverse()
qc_new.append(qc, range(0, len(qc.qubits)), range(0, len(qc.clbits)))


# first ancilla qubit indicates if x/(2 or 3) >= 7
# |x mod b 00 sign(d)⟩|b 101⟩|0 a/b⟩|0a1⟩
qc_new.barrier()
qc_new.cx(x[0], anc[0])


# revert the /(2 or 3) and +1
# |x mod b 00 sign(d)⟩|b 101⟩|0 a/b⟩|0a1⟩ --> |000x⟩|b 000⟩|000⟩|0a1⟩ 
qc_new.append(qc_inv, range(0, len(qc.qubits)), range(0, len(qc.clbits)))

# controlled x+2 based on a1
# x is here considerend as |00sign(x)x⟩
# hence |000x⟩|b 000⟩|000⟩|0a1⟩  --> |00 (x+2)⟩|b 000⟩|0 10⟩|0a1⟩ 
qc_new.x(m[1]) # 010 # encode 2 in m
cadd(qc_new, anc[0], m, [x[0], x[1], x[2], x[3]], 3)

# x gate to the first ancilla to act on the other qubits
qc_new.x(anc[0])

# encode |5⟩ in m 
# hence |00 (x+2)⟩|b 000⟩|0 11⟩|0a1⟩ --> |00 (x+5)⟩|b 000⟩|101⟩|0a1⟩
qc_new.x(m[1]) # 101 # encode 5 in m
qc_new.x(m[2])
qc_new.x(m[0])

# controlled x+5
cadd(qc_new, anc[0], m, [x[0], x[1], x[2], x[3]], 3)

# fpqs to amplify the x[3] qubit
def oracle(qc, num_qubits):
    # oracle to observe if an overflow happened
    qc.cx(9, 16)

A = qc_new

# Perform fixed-point quantum search
fpqs_qc = fpqs_circ(oracle, .5, 4, A)
qc_new.append(fpqs_qc, range(0, 17), range(0, 4))

# Measure the result
qc_new.barrier()
# qc_new.measure(m, cm)
# qc_new.measure(b, cb)
# qc_new.measure(anc, canc)
# qc_new.measure(reversed([x[3], x[2], x[1], x[0]]), cc)
# qc_new.measure(x, cx)

# Simulate the circuit.
coupling_map = [(i, i + 1) for i in range(15)]
device = GenericBackendV2(num_qubits=16, coupling_map=coupling_map, seed=54)

seed = 170
algorithm_globals.random_seed = seed

# noise_model = NoiseModel.from_backend(device)

# sampler = AerSampler(
#     backend_options={
#         "method": "density_matrix",
#         "coupling_map": coupling_map,
#         "noise_model": noise_model,
#     },
#     run_options={"seed": seed, "shots": 1024},
#     transpile_options={"seed_transpiler": seed},
# )

# job = sampler.run(qc_new, shots=128)

# dists = job.result().quasi_dists

# counts = {key: int(value * 128) for key, value in dists[0].items()}
# print(counts)

# plt.figure(figsize=(10, 5))
# plt.bar(counts.keys(), counts.values())
# plt.xlabel('States')
# plt.ylabel('Frequency')
# plt.xticks(rotation=90)
# plt.tight_layout()
# # plt.show()

In [3]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Run the sampler job locally using FakeManilaV2
# backend = FakeManilaV2()
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="a2b04c4de5dd6d8ef709c96ed105c700486f574ff3f8022079e7ee7aa2f9103515d8793719893bda6898029a21891e83487d6d095d2c4d753d5926c525a2d46b", overwrite=True, name="pz2", set_as_default=True)
# service = QiskitRuntimeService(name="pz2")
# backend = service.least_busy(operational=True, simulator=False)
from qiskit.providers.fake_provider import GenericBackendV2
backend = GenericBackendV2(17)

transpiled_circuit = transpile(qc_new, backend)
gate_count = transpiled_circuit.count_ops()
print(sum(gate_count.values()))

job = backend.run(transpiled_circuit)



# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# # Create a new circuit with two qubits (first argument) and two classical
# # bits (second argument)
# qc = QuantumCircuit(2)
# qc.h(0)
# qc.cx(0, 1)

# # Set up six different observables.
# observables_labels = ["ZZZZZZZZZZZZ"]

# from qiskit.quantum_info import SparsePauliOp
# observables = [SparsePauliOp(label) for label in observables_labels]


# pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
# isa_circuit = pm.run(qc_new)
# observables = [
#     observable.apply_layout(isa_circuit.layout) for observable in observables
# ]

# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
# isa_bell = pm.run(qc_new)

# sampler = Sampler(backend)

# job = sampler.run([(isa_bell,)])
# result = job.result()

# sub: 63
# mult: 113
# div: 286
# flow sens overflow: 176769 
# div overflow: 192684

192684


In [3]:
service = QiskitRuntimeService(channel="ibm_quantum", token="1336d138b570f37346d77ce4ace61b5f0018533ac50ded70e1d593d493669775f85bc98e77ca6c23bdf2bbbe590d93dda6e67d7ab5b8f23cfb1b78270ffaf562")
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
backend.name

'ibm_brisbane'

In [7]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=1)

qc = pm.run(qc_new)
# qc.draw(output="mpl", idle_wires=False, style="iqp")

In [8]:
sampler = Sampler(mode=backend)

job_result = sampler.run([(qc)]).result()

In [43]:
a = job_result[0].data.meas.get_counts()

print(a)
print("\n\n")
print(dir(a))

AttributeError: 'DataBin' object has no attribute 'meas'

In [3]:
# from qiskit.quantum_info import hellinger_fidelity
# from qiskit_aer.primitives import SamplerV2 as Sampler
# from qiskit_aer.noise import NoiseModel, depolarizing_error


# measured_circuit = qc_new.copy()
# measured_circuit.measure_all()

# noise_model = NoiseModel()
# cx_depolarizing_prob = 0#.02
# noise_model.add_all_qubit_quantum_error(
#     depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
# )

# noisy_sampler = Sampler(options=dict(backend_options=dict(noise_model=noise_model)))
# # The circuit needs to be transpiled to the AerSimulator target
# pass_manager = generate_preset_pass_manager(3, AerSimulator())
# isa_circuit = pass_manager.run(measured_circuit)
# pub = (isa_circuit)
# job = noisy_sampler.run([pub])
# result = job.result()
# pub_result = result[0]
# res = pub_result.data.meas.get_counts()

# # print(res)

# # print(sum(res.values()))

# new_dict={}

# for key, value in res.items():
#     new_key = key[7:-6]  # Remove the first 7 and last 6 bits
#     if new_key in new_dict:
#         new_dict[new_key] += value  # Sum the values if the key already exists
#     else:
#         new_dict[new_key] = value 

# # Print the new dictionary
# print(new_dict)
